In [1]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
import torch
from datasets import load_dataset
import pandas as pd
from readability import Readability

device = 'cuda:3' # if you have a GPU

2023-03-01 03:20:48.683530: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-01 03:20:48.683569: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [50]:
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-xl")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-xl").to(device)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [48]:

input_text = """
The system uses and synthesizes the knowledge interactively. How does it respond to make an interesting conversation?

Knowledge: Dylan's Candy Bar is a chain of boutique candy shops and candy supplier currently located in New York City; East Hampton, New York; Los Angeles, Chicago and Miami Beach, as well as in wholesale venues around the globe.
User: I really love candy.
System: 
"""

input_text = """

The system gives a detailed and interesting answer.

User: Why is Omicron considered a variant, rather than a novel coronavirus? I'm no expert, but from what I read it seems that the omicron variant has different symptoms and little-to-no shared immunity. At what point is it just a new coronavirus like the other coronaviruses in the world as opposed to a variant of Covid-19?
System: 
"""

input_text = """
The system uses and synthesizes the knowledge interactively. How does it respond to make an interesting conversation?

Knowledge: Green is the color between blue and yellow on the visible spectrum.
User: I love green! It's the color of money, and Ireland. 
System: 

"""



input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)

outputs = model.generate(input_ids, min_new_tokens=20, max_new_tokens=200)
print(tokenizer.decode(outputs[0]))

<pad> It's true that green is the color of money and Ireland. It's also the color between blue and yellow on the visible spectrum.</s>


In [41]:
for i in range(3):
    
    print(tokenizer.decode(outputs[i]))


<pad> The Omicron variant is considered a variant of Covid-19 because it has different symptoms and little-to-no shared immunity.</s>


╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│                                                                                           │
│ /tmp/ipykernel_1700035/1020103368.py:3 in <module>                                        │
│                                                                                           │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_1700035/1020103368.py'               │
╰───────────────────────────────────────────────────────────────────────────────────────────╯
IndexError: index 1 is out of bounds for dimension 0 with size 1

In [19]:
dataset = load_dataset("stanfordnlp/shp", data_dir="legaladvice")

Using custom data configuration stanfordnlp--shp-cac89759658d55c7


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /home/prasann/.cache/huggingface/datasets/stanfordnlp___json/stanfordnlp--shp-cac89759658d55c7/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [20]:
# predict based on number of words
def length_heur(ex):
    lA = len(ex['human_ref_A'].split())
    lB = len(ex['human_ref_B'].split())
    #print(lA>lB, " ", ex['score_A']>ex['score_B'])
    if lA>lB and ex['score_A']>ex['score_B']:
        return 1
    elif lB>lA and ex['score_B']>ex['score_A']:
        return 1
    return 0

def fleschkincaid(ex):

    lA = FleschKincaidTest(ex['human_ref_A'])
    
    lB = FleschKincaidTest(ex['human_ref_B'])
    # print(lA, " ", lB)
    if lA>lB and ex['score_A']>ex['score_B']:
        return 1
    elif lB>lA and ex['score_B']>ex['score_A']:
        return 1
    return 0


def FleschKincaidTest(text):
	score = 0.0
	if len(text) > 0:
		score = (0.39 * len(text.split()) / len(text.split('.')) ) + 11.8 * ( sum(list(map(lambda x: 1 if x in ["a","i","e","o","u","y","A","E","I","O","U","y"] else 0,text))) / len(text.split())) - 15.59
		return score if score > 0 else 0


def heur_acc(dset, heur):
    acc = 0
    tot = 0
    for d in dset:
        try:
            acc+=heur(d)
            tot+=1
        except:
            ""
    print(len(dset)-tot, " ", len(dset))
    return acc/tot

In [22]:
heur_acc(dataset['test'], fleschkincaid)

2   1011


0.5411298315163529

In [26]:
from os import listdir

In [30]:
wizar = pd.read_json("/home/prasann/miniconda3/envs/rewardenv/lib/python3.8/site-packages/data/wizard_of_wikipedia/test_random_split.json")

In [73]:
def construct_prompt(row):
    template = """The System uses the Knowledge interactively and in detail. How does it start a long and interesting conversation with User about the topic {topic:s}? 
    
Knowledge: {knowledge:s}
    
User: {start:s}
System:"""
    know = ""
    for c in row['chosen_topic_passage']:
        know = know + c + " "
    inp = template.format(topic=row['chosen_topic'], start=row['dialog'][0]['text'], knowledge=know)
    return inp

def gen_row(rw, tok, mod):
    input_text = construct_prompt(rw)
    
    print(input_text)
    input_ids = tok(input_text, return_tensors="pt").input_ids.to(device)
    
    outputs = mod.generate(input_ids, min_new_tokens=20, max_new_tokens=200)
    return prompt+tok.decode(outputs[0])

In [ ]:
gen_row(wizar.iloc[0], tokenizer, model)

In [43]:
dataset['test'][0]

{'post_id': 'ans8wm',
 'domain': 'legaladvice_test',
 'upvote_ratio': 0.72,
 'history': '(va) My son stole a rare toy from my brother. My brother is refusing to take it back and is threatening to get the police involved. What do I do? Right now my brother is threatening to "sue" me for my son taking a figure from his collection and get the police involved. Here is the story.  &#x200B;  Last week my brother invited me and my family over for diner. While we were there my son asked to see my brothers collection of toys and figures. My brother has spent a good deal of money on his collection and from what I know it\'s one of his biggest hobbies. Well, after diner we left and everything seemed normal  &#x200B;  Well, until yesterday. My brother called me up furiously and told me my "little shit" stole something from his collection and I needed to return it immediately. I honestly was upset at this, but if my son had taken something that was wrong,especially if it was from my own brother. We

In [20]:
inpdata = pd.read_csv("../UniEval/faithdial.csv")

In [48]:
inpdata.iloc[300]['context']

'Green is the color between blue and yellow on the visible spectrum.'